### Todo:
- ~~Get all possible permutations of a replacement schedule (with 1 replacement, 2, 3)
- make replacement schedules flexible with more than 15 year timelines (i.e. more than 3 replacements)
- ~~vehicle age matrix~~
- ~~vehicle_mileage matrix
- ~~odometer matrix
- ~~acquisition_cost matrix
    - (pct change factor)
- ~~consumables_cost matrix
- ~~maintenance_cost matrix
- ~~figure out how to flag infeasible schedules
- ~~make toy example with two vehicles

# goals tonight 
- ~~figure out how to remove infeasible schedules b4 model
- start using indexes
- ~~add penalty for going over budget or under emissions
- add number of charging stations to build and number of charging stations to operate as vars
- Do a whole round of scrutiny to make this code much more flexible. 
- ~~type up formulation
- start pulling in real data

In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
np.set_printoptions(edgeitems=15,linewidth=600)
# np.core.arrayprint._line_width = 400

In [2]:
start = time.time()

In [3]:
numVehicles = 20

### Schedules 

In [4]:
#todo: will likely need a generalizable method for computing the maximum number of replacements possible
oneReplacement = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
twoReplacements = np.array([1,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
threeReplacements = np.array([1,1,1,0,0,0,0,0,0,0,0,0,0,0,0])

oneReplacement = list(multiset_permutations(oneReplacement))
twoReplacements = list(multiset_permutations(twoReplacements))
threeReplacements = list(multiset_permutations(threeReplacements))

replacementSchedules = np.array(oneReplacement+twoReplacements+threeReplacements)
# replacementSchedules = np.array(twoReplacements+threeReplacements)

In [5]:
replacementSchedules.shape

(575, 15)

In [6]:
numSchedules,_ = replacementSchedules.shape

In [7]:
replacementSchedules = np.repeat(replacementSchedules[np.newaxis,:, :], numVehicles, axis=0)

In [8]:
#opposite of replacement -- may want to use this at some point
keepSchedules = (replacementSchedules-1)*-1
# keepSchedules#[50:80]

### Age

In [9]:
def initialize_vehicle_age(keepSchedules,startingAge=0):
    """Gets the vehicle age building process started. Produces a matrix for the age of the vehicle according to the replacement schedule, which is later fixed by get_vehicle_age"""
    age = startingAge+np.cumsum(keepSchedules,axis=2)*keepSchedules
    age[age==startingAge] = 0 #fixes the fact that replaced vehicles start at 0 (if this wasn't here they would start at the starting age)
    return age


def get_vehicle_age(age=None,k=1,startingAge=0):
    if k==1:
        age = initialize_vehicle_age(keepSchedules,startingAge)

    diff = np.diff(age,axis=2)
    diffMask = np.append(np.ones(shape=(numVehicles,numSchedules,1)),diff,axis=2)>1
    age[diffMask]=k

    if age[diffMask].size==0:
        return age
    else:
        return get_vehicle_age(age,k=k+1)

age = get_vehicle_age(startingAge=0)#[50:80]

In [10]:
age.shape

(20, 575, 15)

### Mileage

In [11]:
#vehicle_mileage matrix 
# vehicle_mileage = np.repeat(np.round(np.random.normal(loc=10000,scale=2000,size=(1, 15))),numSchedules,axis=0)
annual_mileage = np.ones(shape=(numVehicles,numSchedules,15))
annual_mileage[0,:,:]*=np.round(np.random.normal(loc=10000,scale=2000))
annual_mileage[1,:,:]*=np.round(np.random.normal(loc=11000,scale=2000))

In [12]:
#matrix showing what the odometer reading will be under each schedule
odometer = annual_mileage*age

### Acquisition

In [13]:
#Todo: need to look up acquisitoin cost for each vehicle
def get_acquisition_cost(replacementSchedules):
    acquisition = replacementSchedules.copy()
    for v in range(numVehicles):
        acquisition[v,:,:]*=np.random.randint(20000,40000)
    return acquisition

acquisition = get_acquisition_cost(replacementSchedules)

### Consumables

In [14]:
def get_vehicle_type_trackers(replacementSchedules):
    """Returns two matrices. One that tracks if in a givemn year a schedule implies the vehicle is still an ice, and then the opposite: whether or not in a given year a vehicle is now an EV"""
    firstReplacements = np.argmax(replacementSchedules[0]==1,axis=1) #gets index of year the vehicle is first replaced (ie it transitions from ICE to EV)
    is_ice = replacementSchedules.copy()
    is_ev = replacementSchedules.copy()
    
    for i in range(0,numSchedules): #there is most definitely a better way to do this in numpy but I took way too long researching
        is_ice[:,i,firstReplacements[i]:] = 0
        is_ice[:,i,:firstReplacements[i]] = 1

        is_ev[:,i,firstReplacements[i]:] = 1
        is_ev[:,i,:firstReplacements[i]] = 0
    return is_ice,is_ev

is_ice,is_ev = get_vehicle_type_trackers(replacementSchedules)

In [15]:
def get_consumables(is_ice,is_ev):
    """Will make this function more flexible later. Calculates fuel cost as if always ICE and always EV. And then applies to the schedules based on when the initial transition from ICE to EV occurs. """   
    #ICE
    #fuel $ = mileage/mpg*cpg
    cpg = 2.25
    mpg = 22
    fuel = annual_mileage/mpg*cpg

        #EV
    #fuel $ = mileage/mpeg * cpeg
    cpeg = 1.2
    mpeg = 100
    electricity = annual_mileage/mpeg*cpeg

    consumables = np.round(fuel*is_ice+(electricity*is_ev))
    return consumables

consumables = get_consumables(is_ice,is_ev)

### Maintenance cost

In [16]:
def get_maintenance_cost(age,annual_mileage,odometer):
    """- ! because this is likely to change. For now I'm just going to treat as a linear regression with made up coeffs."""
    age_coef = .01
    mileage_coef = .2
    odometer_coef = .1
    maintenance = (age_coef*age)+(mileage_coef*annual_mileage)+(odometer_coef*odometer)
    return maintenance

maintenance = get_maintenance_cost(age,annual_mileage,odometer)

### Emissons

In [17]:
def get_emissions(is_ice,is_ev):
    """Will make this function more flexible later. Calculates fuel cost as if always ICE and always EV. And then applies to the schedules based on when the initial transition from ICE to EV occurs. """   
    #calc: kg CO2/gallon * mileage/mpg
    
    ice_emission_factor = 2.421
    mpg = 22
    ice_emissions = ice_emission_factor*annual_mileage/mpg

    ev_emission_factor = 0
    mpge = 100
    ev_emissions = ev_emission_factor*annual_mileage/mpge

    emissions = np.round((ice_emissions*is_ice)+(ev_emissions*is_ev))
    return emissions

emissions = get_emissions(is_ice,is_ev)

### Find and filter out infeasible schedules
- infeasible in the sense that they have a replacement happening in years where the vehicle is both under 6 years old and doesn't yet have 150k miles on it

In [18]:
odometer_diff = np.diff(odometer)
odometer_check = (odometer_diff>-150000) & (odometer_diff<=0)

In [19]:
age_diff = np.diff(age)
age_check = (age_diff>-6) & (age_diff<=0)#.any()


In [20]:
both_check = odometer_check*age_check

In [21]:
infeasible_filter = both_check.any(axis=2)

In [22]:
# replacementSchedules[~infeasible_filter]

In [23]:
#!I feel like there should be more feasible schedules. Will need to come back to this. 
def find_infeasible_schedules(odometer,age):
    """Generates a mask that is True for any schedule that is infeasible. These can be filtered out before running the model."""
    odometer_diff = np.diff(odometer)
    odometer_check = (odometer_diff>-150000) & (odometer_diff<=0)

    age_diff = np.diff(age)
    age_check = (age_diff>-6) & (age_diff<=0)#.any()

    both_check = odometer_check*age_check
    is_infeasible = both_check.any(axis=2)
    return is_infeasible

infeasible_filter = find_infeasible_schedules(odometer,age)
# replacementSchedules[~infeasible_filter]

In [24]:
infeasible_filter.shape

(20, 575)

### Toy Model 

In [25]:
import gurobipy as grb

In [26]:
num_vehicles,num_schedules,num_years = replacementSchedules.shape

In [27]:
vehicles = [v for v in range(0,num_vehicles)]
schedules = [s for s in range(0,num_schedules)]
years = [t for t in range(0,num_years)]
finalYear = max(years)

In [28]:
c = {}
a = {}
m = {}
e = {}
for v in vehicles:
    for s in schedules:
        if not infeasible_filter[v,s]:
            c[v,s] = consumables[v,s]
            a[v,s] = acquisition[v,s]
            m[v,s] = maintenance[v,s]
            e[v,s] = emissions[v,s]

In [29]:
consumables = c.copy()
acquisition = a.copy()
maintenance = m.copy()
emissions = e.copy()

In [30]:
#! I think I can pull the code I used in modelinputsgen
budget_acquisition = 1300000*np.ones(shape=(15)) 
budget_operations = 1000000*np.ones(shape=(15))

emissions_goal = 40000

numDesiredSolutions = 3

In [31]:
try: 
    m.reset()
    del m    
except:
    None
    
m = grb.Model('carnet')

Using license file C:\Users\elynch\gurobi.lic
Academic license - for non-commercial use only


In [32]:
m.setParam('PoolSearchMode',2) #tell gurobi I want multiple solutions
m.setParam('PoolSolutions',numDesiredSolutions) #number of solutions I want
m.setParam('TimeLimit',30)

Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolSolutions to 3
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [33]:
x = m.addVars(vehicles,schedules,vtype=grb.GRB.BINARY,name='x')
penalty_budget = m.addVar(vtype=grb.GRB.CONTINUOUS,name='penalty_budget')
penalty_emissions = m.addVar(vtype=grb.GRB.CONTINUOUS,name='penalty_emissions')
# y = m.addVars(years,)

In [34]:
w = {'cost':0.70,'emissions':0.30}

In [35]:
# total_cost = np.sum(consumables+acquisition+maintenance,axis=2)
# total_acquisition_cost = np.sum(acquisition,axis=2)
# total_operations_cost = np.sum(consumables+maintenance,axis=2)

In [36]:
validSchedules = list(consumables.keys())

In [37]:
obj = m.setObjective(grb.quicksum(w['cost']*consumables[v,s][t]*x[v,s] for v,s in validSchedules for t in years) + 
                     grb.quicksum(w['emissions']*emissions[v,s][finalYear]*x[v,s] for v,s in validSchedules) +
                     1000000*(penalty_budget+penalty_emissions),grb.GRB.MINIMIZE)
# obj = m.setObjective(grb.quicksum(w['cost']*(consumables[v,s,t]+acquisition[v,s,t]+maintenance[v,s,t])*x[v,s] + 
#                                   w['emissions']*emissions[v,s,finalYear]*x[v,s] for v in vehicles for s in schedules for t in years),grb.GRB.MINIMIZE)

In [38]:
import pandas as pd
validSchedulesPerVehicle = pd.DataFrame(validSchedules).groupby(0)[1].unique().to_dict()

In [39]:
c1 = m.addConstrs((grb.quicksum(x[v,s] for s in validSchedulesPerVehicle[v])==1 for v in vehicles),'one_schedule_per_vehicle')

In [40]:
c2 = m.addConstrs((grb.quicksum((consumables[v,s][t]+maintenance[v,s][t])*x[v,s] for v,s in validSchedules) <= budget_operations[t]+penalty_budget for t in years),'operations_budget')
c3 = m.addConstrs((grb.quicksum(acquisition[v,s][t]*x[v,s] for v,s in validSchedules) <= budget_operations[t]+penalty_budget for t in years),'acquisition_budget')

In [41]:
c4 = m.addConstr((grb.quicksum(emissions[v,s][finalYear]*x[v,s] for v,s in validSchedules) <= emissions_goal+penalty_emissions),'emissions_goal')

In [42]:
# c5 = m.addConstrs((infeasible_filter[v,s]*x[v,s] <= 0 for v in vehicles for s in schedules),'infeasible_schedules')

In [43]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 51 rows, 11502 columns and 7771 nonzeros
Model fingerprint: 0x8de1fb7b
Variable types: 2 continuous, 11500 integer (11500 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+04]
  Objective range  [1e+03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 6295.8000000
Presolve removed 31 rows and 0 columns
Presolve time: 0.00s
Presolved: 20 rows, 11502 columns, 440 nonzeros
Variable types: 2 continuous, 11500 integer (11500 binary)
Found heuristic solution: objective 2541.0000000

Root relaxation: objective 2.541000e+03, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0          -    0      2541.00000 2541.00000  0.00%     -    0s

Optimal solution found at node 0 - now completing solution pool...

  

In [44]:
end = time.time()
print("--- %s seconds ---" % (time.time() - start))


--- 1.0826854705810547 seconds ---


In [46]:
#multiple solutions
solution = []
options = ['A','B','C']
for solution in range(0,299):
    print()
    print(solution)
#     print(f'Option: {options[solution]}')
    m.setParam('SolutionNumber',solution)
    for v in vehicles:
        for s in schedules:
            if x[v,s].xn==1:
                print(f'   Vehicle: {v+1} Schedule: {s} {replacementSchedules[v,s]}')     


0
   Vehicle: 1 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 2 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 3 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 4 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 5 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 6 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 7 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 8 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 9 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 10 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 11 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 12 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 13 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 14 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 15 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 16 Schedule: 505 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
   Vehicle: 17 Schedule: 505 [

AttributeError: Unable to retrieve attribute 'xn'

In [52]:
print(f'age: {age[0,505]}')
print()
print(f'age: {age[0,505]}')
print()
print(f'annual_mileage: {annual_mileage[0,505]}')
print()
print(f'odometer: {odometer[0,505]}')
print()
print(f'acquisition cost: {acquisition[0,505]}')
print()
print(f'maintenance cost: {maintenance[0,505]}')
print()
print(f'consumables cost: {consumables[0,505]}')
print()
print(f'emissions: {emissions[0,505]}')

age: [0 1 2 3 4 5 6 0 1 2 3 4 5 6 0]

age: [0 1 2 3 4 5 6 0 1 2 3 4 5 6 0]

annual_mileage: [9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906. 9906.]

odometer: [    0.  9906. 19812. 29718. 39624. 49530. 59436.     0.  9906. 19812. 29718. 39624. 49530. 59436.     0.]

acquisition cost: [25360     0     0     0     0     0     0 25360     0     0     0     0     0     0 25360]

maintenance cost: [1981.2  2971.81 3962.42 4953.03 5943.64 6934.25 7924.86 1981.2  2971.81 3962.42 4953.03 5943.64 6934.25 7924.86 1981.2 ]

consumables cost: [119. 119. 119. 119. 119. 119. 119. 119. 119. 119. 119. 119. 119. 119. 119.]

emissions: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
len(m.x)

In [ ]:
emissions[7]

In [ ]:
consumables[7]

In [ ]:
maintenance[7]

### Old stuff from vehicle age recursion 